In [7]:
import json

In [8]:

def load_data(filename: str) -> list:
    data_arr = []
    with open(filename, encoding="utf8") as file:
        for line in file:
            data = json.loads(line)
            data_arr.append(data)

    return data_arr

In [9]:
def get_exception_types(data_arr: list) -> dict:
    result_dict = {
        "timed out": 0
    }
    num_test = 0
    for data in data_arr:
        for sample in data["results"]:
            for result in sample:
                if not result["passed"]:
                    if "exception" not in result:
                        result_dict[result["result"]] += 1
                    elif result["exception"] not in result_dict:
                        result_dict[result["exception"]] = 0
                    else:
                        result_dict[result["exception"]] += 1
                    
                num_test += 1

    return result_dict

In [10]:
def get_task_stats(data_arr: list) -> dict:
    task_dict = dict()
    overall_total_tests = 0
    overall_passed_tests = 0

    for data in data_arr:
        total_tests = 0
        passed_tests = 0
        for stats in data["statistics"]:
            total_tests += stats["total_tests"]
            passed_tests += stats["passed_tests"]
        
        overall_total_tests += total_tests
        overall_passed_tests += passed_tests
        success_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        task_dict[data["task_id"]] = {
            "total_tests": total_tests,
            "passed_tests": passed_tests,
            "success_rate": success_rate
        }

    task_dict["summary"] = {
        "total_tests": overall_total_tests,
        "passed_tests": overall_passed_tests,
        "success_rate": (overall_passed_tests / overall_total_tests * 100) if overall_total_tests > 0 else 0,
        "average_tests_per_sample": round(overall_total_tests / len(data_arr), 2) if len(data_arr) > 0 else 0
    }

    return task_dict

In [11]:
def get_cwe_stats(data_arr: list) -> dict:
    cwe_dict = dict()
    for data in data_arr:
        total_tests = 0
        passed_tests = 0
        for stats in data["statistics"]:
            total_tests += stats["total_tests"]
            passed_tests += stats["passed_tests"]
        
        if data["cwe_id"] not in cwe_dict:
            cwe_dict[data["cwe_id"]] = {
                "total_tests": total_tests,
                "passed_tests": passed_tests,
            }
        else:
            cwe_dict[data["cwe_id"]]["total_tests"] += total_tests
            cwe_dict[data["cwe_id"]]["passed_tests"] += passed_tests

    for cwe_id, item in cwe_dict.items():
        success_rate = (item["passed_tests"] / item["total_tests"] * 100) if item["total_tests"] > 0 else 0
        cwe_dict[cwe_id]["success_rate"] = success_rate

    return cwe_dict

In [12]:
from pathlib import Path  # import Path from pathlib module

directory = Path('eval-go')  # set directory path

result_dict = dict()
for file in directory.iterdir():  
    if file.is_file() and file.suffix == ".jsonl":  # Check if it's a file
        data_arr = load_data(file)
        # err_dict = get_exception_types(data_arr)
        task_dict = get_task_stats(data_arr)
        cwe_dict = get_cwe_stats(data_arr)

        result_dict[Path(file).stem] = {
            # "err_stats": err_dict,
            "task_stats": task_dict,
            "cwe_stats": cwe_dict
        }

In [13]:
with open("summarized-go.json", "w") as f:
    f.write(json.dumps(result_dict, indent=4))